In [1]:
# 필요한 라이브러리 설치
!pip install langchain-community langchain-chroma langchain-openai bs4

# 필요한 라이브러리 임포트
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [32]:
# OpenAI 모델 설정 (gpt-4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini", api_key="")

# 웹 로더 설정 - 인코딩 설정 추가
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-18vt64m") # 스파르타 블로그의 본문 클래스
        )
    ),
    encoding="utf-8"  # 인코딩 명시적으로 지정. 안하면 깨짐
)


# 문서 로드할 때 인코딩 확인을 위한 출력 추가
docs = loader.load()
print("로드된 문서 내용 미리보기:")
print(docs[0].page_content[:100])  # 처음 100자 출력해보기
# 문서 로드
docs = loader.load()


로드된 문서 내용 미리보기:
스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  738·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어문학을 공부하며 학습 속도도 눈에 띄게 빨라질 수 있어요.

In [34]:
# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=""))

# 검색기(retriever) 생성 - 여기에 검색 개수 증가 코드 적용
# 다양성을 고려한 검색
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 15, "fetch_k": 20}
)
# 사용자 질문에 대한 문서 검색
retrieved_docs = retriever.invoke("ALL-in 코딩 공모전 수상작들을 요약해줘")

# 검색된 문서와 질문을 LLM에 전달
custom_prompt = """당신은 RAG(검색 증강 생성) 시스템입니다. 
다음은 ALL-in 코딩 공모전에 대한 정보입니다:

{context}

요청: {question}

중요 지침:
1. 위 정보를 바탕으로 ALL-in 코딩 공모전의 모든 수상작들을 요약해주세요.
2. 각 수상작의 이름, 등급(대상/우수상/장려상), 개발자, 주요 기능을 포함해야 합니다.
3. 어떤 수상작도 빠뜨리지 말고 모두 포함해주세요.

답변:"""

# 검색된 문서를 포맷팅하여 컨텍스트로 제공
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

user_prompt = {"context": format_docs(retrieved_docs), "question": "ALL-in 코딩 공모전 수상작들을 요약해줘"}
messages = [{"role": "user", "content": custom_prompt.format(**user_prompt)}]
response = llm.invoke(messages)
print(response.content)

### ALL-in 코딩 공모전 수상작 요약

1. **수상작 이름**: **Lexi Note**
   - **등급**: 대상
   - **개발자**: 다나와(김다애, 박나경)
   - **주요 기능**: 사용자가 단어를 드래그하면 네이버 사전과 연동되어 단어의 의미를 찾아주며 필기 노트 기능을 동시에 제공. 번역 버튼을 이용하면 파파고 번역기로도 연동되어 긴 문장 이해가 용이. 할일 목록 및 스케줄 템플릿을 제공하여 효율적인 언어 학습을 지원.

**추가 수상작 목록이 제공되지 않았으므로, 현재로서는 Lexi Note에 대한 정보만 포함되었습니다.**


In [42]:
# splits에 저장된 문서들 확인..
print("분할된 문서 수:", len(splits))
for i, split in enumerate(splits[1:7]):
    print(f"분할 {i+1} 미리보기:")
    print(split.page_content[:100])
    print("---")

분할된 문서 수: 7
분할 1 미리보기:
이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필
---
분할 2 미리보기:
추가 수입을 얻을 수 있는 기회를 가집니다. 실시간 알림 기능을 통해 매칭이 이뤄지면 즉시 알림을 받을 수 있어 빠르게 문제를 해결할 수 있습니다. 벌레를 퇴치한 후 사용자는 서로
---
분할 3 미리보기:
자신의 의견이나 질문을 쉽게 제출할 수 있어 교수님은 실시간으로 학생들의 이해도를 파악할 수 있습니다. 수업 중 또는 후에 교수자가 수업에 대한 피드백을 받을 수 있는 평가 기능과
---
분할 4 미리보기:
맞는 동아리를 선택할 수 있습니다.사용한 기술 스택 -FE(프론트엔드): Spring Boot, Redis, MySQL-BE(백엔드): SwiftUI Framework, OAuth
---
분할 5 미리보기:
보며 학습 계획도 조정할 수 있어요. <학교생활 매니저>앱을 사용하면 체계적으로 대학 생활을 관리하고, 남은 학업과 일정을 효율적으로 관리할 수 있습니다.사용한 기술 스택 -FE(
---
분할 6 미리보기:
통해 캠퍼스 내에 보다 다양한 문화가 공존하고, 학생들 간의 유대감이 강화되고, 서로 다른 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소
---


In [37]:
# 검색된 문서 내용 확인
print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- 검색된 문서 {i+1} ---")
    print(doc.page_content)
    print("----------------")

검색된 문서 수: 15

--- 검색된 문서 1 ---
스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  736·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.
----------------

--- 검색된 문서 2 ---
스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  736·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.
----------------

--- 검색된 문서 3 ---
스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  736·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의